じゃらんスクレイピング

In [112]:
import requests
from bs4 import BeautifulSoup
import time
import sqlite3

# 基本URL
base_url = "https://www.jalan.net/kankou/130000/page_{}/?screenId=OUW1702"

# ヘッダーを設定（スクレイピング対策を回避するため）
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# 店舗情報を格納するリスト
data_list = []

# ページを巡回してデータを取得
def scrape_page(page_number):
    url = base_url.format(page_number)
    response = requests.get(url, headers=headers)

    # 自動的に適切なエンコーディングを設定し、デコード時にエラーを処理
    response.encoding = response.apparent_encoding
    content = response.text.encode('utf-8', errors='replace').decode('utf-8', errors='replace')

    # BeautifulSoupでHTMLを解析
    soup = BeautifulSoup(content, "html.parser")

    # デバッグ: HTMLスニペット（該当セクションを絞り込む）
    main_section = soup.find("ul", class_="cassetteList-list")
    if not main_section:
        print(f"Page {page_number}: 該当セクションが見つかりませんでした。")
        return []

    # 観光地要素を探す
    item_elements = main_section.find_all("li", class_="item")
    results = []

    if item_elements:
        print(f"Page {page_number}: {len(item_elements)} 件の観光地要素を発見。")
        for item in item_elements:
            try:
                # タイトルの取得
                title_tag = item.find("h3", class_="item-name")
                title = title_tag.find("a").get_text(strip=True) if title_tag else "タイトルなし"

                # 星評価の取得（数値型に変換）
                rating_tag = item.find("span", class_="reviewPoint")
                rating_text = rating_tag.get_text(strip=True) if rating_tag else "0"
                rating = float(rating_text) if rating_text.replace(".", "").isdigit() else 0.0

                # 口コミ件数の取得（数字のみ）
                reviews_tag = item.find("span", class_="reviewCount")
                reviews_text = reviews_tag.get_text(strip=True) if reviews_tag else "0"
                reviews = int("".join(filter(str.isdigit, reviews_text))) if reviews_text else 0

                # データ検証: 必要な情報が揃っている場合のみ追加
                if title != "タイトルなし" and rating > 0 and reviews > 0:
                    results.append({"title": title, "rating": rating, "reviews": reviews})
            except AttributeError as e:
                print(f"要素の取得に失敗しました: {e}")
    else:
        print(f"Page {page_number}: 観光地要素が見つかりませんでした。")

    return results

# ページを巡回してデータを収集
for page in range(1, 21):  # 1ページ目から7ページ目まで巡回
    results = scrape_page(page)
    if not results:
        print(f"Page {page}: データが見つかりませんでした。処理を終了します。")
        break
    data_list.extend(results)
    time.sleep(5)  # ページ取得間に5秒待機

# SQLiteにデータを格納
conn = sqlite3.connect("tokyo_kankou.db")
cursor = conn.cursor()

# テーブルを再作成（既存のテーブルを削除）
cursor.execute("DROP TABLE IF EXISTS kankou")
cursor.execute('''
CREATE TABLE kankou (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    title TEXT,
    rating REAL,
    review_count INTEGER
)
''')

# データを挿入
for data in data_list:
    cursor.execute('''
    INSERT INTO kankou (title, rating, review_count) VALUES (?, ?, ?)
    ''', (data["title"], data["rating"], data["reviews"]))

# コミットして接続を閉じる
conn.commit()
conn.close()

print("取得した観光地情報をSQLiteデータベースに保存しました。")

Page 1: 95 件の観光地要素を発見。
Page 2: 70 件の観光地要素を発見。
Page 3: 69 件の観光地要素を発見。
Page 4: 71 件の観光地要素を発見。
Page 5: 75 件の観光地要素を発見。
Page 6: 66 件の観光地要素を発見。
Page 7: 65 件の観光地要素を発見。
Page 8: 58 件の観光地要素を発見。
Page 9: 54 件の観光地要素を発見。
Page 10: 52 件の観光地要素を発見。
Page 11: 38 件の観光地要素を発見。
Page 12: 77 件の観光地要素を発見。
Page 13: 39 件の観光地要素を発見。
Page 14: 68 件の観光地要素を発見。
Page 15: 44 件の観光地要素を発見。
Page 16: 33 件の観光地要素を発見。
Page 17: 32 件の観光地要素を発見。
Page 18: 75 件の観光地要素を発見。
Page 19: 32 件の観光地要素を発見。
Page 20: 38 件の観光地要素を発見。
取得した観光地情報をSQLiteデータベースに保存しました。


In [116]:
# SQLiteデータベースに接続
conn = sqlite3.connect("tokyo_kankou.db")
cursor = conn.cursor()

# ランキングスコアを計算して降順でデータを取得
cursor.execute('''
SELECT *, (rating * 10 + review_count * 0.1) AS score 
FROM kankou 
ORDER BY score DESC
''')
rows = cursor.fetchall()

# 結果を表示
print("\nユーザー評価に基づいたランキング（スコア降順）:")
for row in rows:
    print(f"{row}")

conn.close()

print("データベース操作が完了しました。")


ユーザー評価に基づいたランキング（スコア降順）:
(13, '浅草寺', 4.2, 7667, 808.7)
(28, '東京駅', 4.1, 6948, 735.8000000000001)
(54, '東京ソラマチ', 4.1, 5150, 556.0)
(56, 'アメ横', 3.9, 4592, 498.20000000000005)
(47, '上野動物園', 4.2, 4448, 486.8)
(30, '羽田空港（東京国際空港）', 4.2, 4141, 456.1)
(59, '明治神宮', 4.2, 4123, 454.3)
(383, '原宿竹下通り', 3.8, 3734, 411.40000000000003)
(43, '築地場外市場', 4.1, 3691, 410.1)
(84, 'レゴランド・ディスカバリー・センター東京', 4.5, 3451, 390.1)
(58, '東京タワー', 4.2, 3369, 378.90000000000003)
(68, '東京ドーム', 4.1, 2923, 333.3)
(91, '新宿御苑', 4.2, 2681, 310.1)
(122, '国営昭和記念公園', 4.3, 2670, 310.0)
(103, '伊勢丹新宿店', 4.2, 2629, 304.90000000000003)
(14, 'サンシャイン水族館', 4.1, 2618, 302.8)
(167, '恵比寿ガーデンプレイス', 4.1, 2384, 279.4)
(379, '表参道', 4.1, 2374, 278.4)
(166, '六本木ヒルズ', 4.0, 2344, 274.4)
(433, 'VenusFort', 3.9, 2343, 273.3)
(82, '東京都庁舎展望室', 4.2, 2233, 265.3)
(96, '東京スカイツリー', 4.2, 2221, 264.1)
(15, 'マクセル アクアパーク品川', 4.1, 2157, 256.70000000000005)
(159, '渋谷ヒカリエ', 4.0, 2137, 253.70000000000002)
(162, '三鷹の森ジブリ美術館（三鷹市立アニメーション美術館）', 4.3, 2007, 243.70000000

じゃらん公式サイトの東京観光スポットランキングからスクレイピングで情報を取得しlitesqlに保存
仮定した計算方式をもとにランキングに必要なポイントを計算し昇順で出力